# Exploratory Analysis 

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import os
import sys
import numpy as np

pd.set_option('display.max_columns', 500)

## Load Data

In [2]:
applicationDF = pd.read_csv(os.path.join('.','data','application_train.csv'))

## Basic Exploration

In [3]:
applicationDF.shape

(307511, 122)

In [4]:
applicationDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [5]:
applicationDF.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [6]:
applicationDF['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [7]:
applicationDF['TARGET'].value_counts() / applicationDF.shape[0]

0    0.919271
1    0.080729
Name: TARGET, dtype: float64

## Clean Data

In [8]:
from tools.modelTools import categoricalColumns, numericalColumns

In [9]:
categoricalColumns = categoricalColumns()
numericalColumns = numericalColumns()

In [10]:
# create matrix of missing values
# only keep the column if it has missing values

numericalColumnsNan = (applicationDF[numericalColumns].isnull() * 1)
numericalColumnsNanList = (numericalColumnsNan.sum(axis=0) > 0)
numericalColumnsNanListFilter = numericalColumnsNanList[numericalColumnsNanList==True].index
numericalColumnsNanListFINAL = numericalColumnsNan[numericalColumnsNanListFilter]

In [11]:
# create matrix of missing values
# only keep the column if it has missing values

categoricalColumnsNan = (applicationDF[categoricalColumns].isnull() * 1)
categoricalColumnsNanList = (categoricalColumnsNan.sum(axis=0) > 0)
categoricalColumnsNanListFilter = categoricalColumnsNanList[categoricalColumnsNanList==True].index
categoricalColumnsNanListFINAL = categoricalColumnsNan[categoricalColumnsNanListFilter]

In [ ]:
print(applicationDF[categoricalColumns].shape)
print(numericalColumnsNanListFINAL.shape)
print(categoricalColumnsNanListFINAL.shape)


### Define the quantatative data frame

In [12]:
categoricalDF = pd.concat([applicationDF[categoricalColumns], numericalColumnsNanListFINAL, categoricalColumnsNanListFINAL], axis=1)

In [13]:
targets = applicationDF["TARGET"]

In [14]:
quant_df = pd.concat([applicationDF[numericalColumns], targets], axis=1)

In [ ]:
quant_df.head()

In [ ]:
print(targets[targets==0].count())
print(targets[targets==1].count())

In [15]:
positive_loans = quant_df[quant_df["TARGET"]==1]
positive_loans.shape[0]

24825

In [16]:
positive_loans = positive_loans.append([positive_loans]*10,ignore_index=True)
positive_loans.shape[0]

273075

In [17]:
negative_loans = quant_df[quant_df["TARGET"]==0]
negative_loans.shape[0]

282686

In [18]:
quant_df = positive_loans.append([negative_loans])

In [19]:
# mix the rows
import csv
quant_df = quant_df.sample(frac=1).reset_index(drop=True)
# quant_df.to_csv(path_or_buf='quant_df.csv', quoting=csv.QUOTE_NONE, index=False)

In [20]:
# fill in NaN
quant_df = quant_df.fillna(quant_df.mean())

In [ ]:
quant_df.head()

In [ ]:
quant_df.shape

### Train the model

In [21]:
import keras
from keras import layers, utils, optimizers, losses, models, callbacks
keras.__version__

Using TensorFlow backend.


'2.1.6'

In [ ]:
import tensorflow as tf
tf.__version__

In [22]:
# define the validation dataset
validation_set = quant_df.sample(frac=0.2).reset_index(drop=False)
training_set = quant_df.drop(validation_set["index"], axis=0).reset_index(drop=True)
validation_set = validation_set.drop(["index"], axis=1).reset_index(drop=True)

In [ ]:
validation_set.head()

In [ ]:
training_set.head()

In [23]:
# setup the training dataset
training_targets = training_set["TARGET"]

# drop the old index from the validation dataset
validation_targets = validation_set["TARGET"]

#drop the target columns
training_set = training_set.drop(["TARGET"], axis=1).reset_index(drop=True)
validation_set = validation_set.drop(["TARGET"], axis=1).reset_index(drop=True)

In [24]:
# get the ndarray out of the dataframe
training_set = training_set.values
validation_set = validation_set.values

In [25]:
# onehot for the targets
training_targets = utils.to_categorical(training_targets)
validation_targets_one_hot = utils.to_categorical(validation_targets)

In [ ]:
print(training_set.shape)
print(training_targets.shape)
print(validation_set.shape)
print(validation_targets_one_hot.shape)

In [26]:
# normalize the training and validation data
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
training_set = min_max_scaler.fit_transform(training_set)
validation_set = min_max_scaler.fit_transform(validation_set)

In [27]:
# define the NN architecture
inputs = layers.Input(shape=(69,), name="inputs")

fc1 = layers.Dense(units=256, activation='relu', name='fc1')(inputs)
fc2 = layers.Dense(units=512, activation='relu', name='fc2')(fc1)
fc3 = layers.Dense(units=512, activation='relu', name='fc3')(fc2)
fc4 = layers.Dense(units=512, activation='relu', name='fc4')(fc3)

outputs = layers.Dense(units=2, activation='softmax', name='outputs')(fc4)
model = models.Model(inputs=[inputs], outputs=[outputs])

In [28]:
# set up the optimizer and the loss function
optimizer = optimizers.Adam(lr=0.0001)
loss = losses.binary_crossentropy
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [29]:
model.compile(loss=loss, metrics=['accuracy'], optimizer=optimizer)
model.fit(x=training_set, y=training_targets, batch_size=256, validation_split=0.1, callbacks=[early_stopping], epochs=50)

Train on 400148 samples, validate on 44461 samples
Epoch 1/50
400148/400148 [==============================] - 63s 158us/step - loss: 0.6107 - acc: 0.6685 - val_loss: 0.6070 - val_acc: 0.6686
Epoch 2/50
400148/400148 [==============================] - 64s 160us/step - loss: 0.6041 - acc: 0.6739 - val_loss: 0.6020 - val_acc: 0.6729
Epoch 3/50
345600/400148 [========================>.....] - ETA: 8s - loss: 0.6004 - acc: 0.6768

KeyboardInterrupt: 

In [ ]:
# save the model
model.save('./models/model')

In [30]:
loss_, acc_ = model.evaluate(x=validation_set, y=validation_targets_one_hot)

111152/111152 [==============================] - 16s 148us/step


In [31]:
print("Loss: {}, Accuracy: {}".format(loss_, acc_))

Loss: 0.6034036703601885, Accuracy: 0.6748506549589751


### AUC 

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
y_pred_keras = model.predict(validation_set)
fpr, tpr, thresholds = roc_curve(validation_targets, np.argmax(y_pred_keras, axis=1))
print("AUC: {}".format(roc_auc_score(np.argmax(y_pred_keras, axis=1), validation_targets)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
plt.show()

### Neural Net Output

In [ ]:
submission_data_NN = os.path.join('.','data','application_test.csv')
submission_df_NN_raw = pd.read_csv(submission_data_NN)

submission_df_NN = submission_df_NN_raw[numericalColumns]
submission_ids = submission_df_NN_raw['SK_ID_CURR'].copy()

In [ ]:
submission_df_NN.shape

In [ ]:
submission_ids.shape

In [ ]:
submission_df_NN.fillna(quant_df.mean(), inplace=True)

submission_df_NN_scaled = min_max_scaler.transform(submission_df_NN)

In [ ]:
from keras.models import load_model
model = load_model('./models/model')

In [ ]:
submission_pred_keras = model.predict(submission_df_NN_scaled)

In [ ]:
submission_pred_keras[:,:1]

In [ ]:
submission_df_NN.head()

In [ ]:
finalOutputNN = DataFrame(np.concatenate([submission_ids.values.reshape(-1,1), submission_pred_keras[:,1:]], axis=1))

finalOutputNN.loc[:,0] = finalOutputNN.loc[:,0].astype(int)

finalOutputNN.columns=['SK_ID_CURR','TARGET']

In [ ]:
finalOutputNN.to_csv('submissions/submission.csv', index=False)

In [ ]:
# # if you do not want to return a sparse matrix
# # cat_encoder = OneHotEncoder(sparse=False)

# cat_encoder = m.OneHotEncoder()
# cat_1hot = cat_encoder.fit_transform(categoricalDF)

In [ ]:
# num_pipeline = Pipeline([
#         ('imputer', Imputer(strategy="median")),
#         ('std_scaler', StandardScaler()),
#     ])

In [ ]:
# housing_num_tr = num_pipeline.fit_transform(applicationDF[numericalColumns])

## Prepare Data for Modeling

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(applicationDF, test_size=0.2, random_state=42)

X_train = train_set.drop(['SK_ID_CURR','TARGET'], axis=1) # drop labels for training set
y_train = train_set['TARGET'].copy()

X_test = test_set.drop(['SK_ID_CURR','TARGET'], axis=1) # drop labels for training set
y_test = test_set['TARGET'].copy()

In [ ]:
X_train.shape

In [ ]:
%load_ext autoreload

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer

%aimport tools.modelTools
%autoreload 1

In [ ]:
num_pipeline = Pipeline([
        ('selector', tools.modelTools.DataFrameSelector(numericalColumns)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', tools.modelTools.DataFrameSelector(categoricalColumns, fillna=True)),
        ('cat_encoder', tools.modelTools.OneHotEncoder(sparse=False)),
    ])

In [ ]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [ ]:
X_train.shape

In [ ]:
home_credit_prepared = full_pipeline.fit_transform(X_train)
home_credit_prepared

In [ ]:
home_credit_prepared.shape

## Train Model

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=5, random_state=42)
sgd_clf.fit(home_credit_prepared, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, home_credit_prepared, y_train, cv=3, scoring="accuracy")

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, home_credit_prepared, y_train, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_train_pred)

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train, y_train_pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train, y_train_pred)

In [ ]:
y_scores = cross_val_predict(sgd_clf, home_credit_prepared, y_train, cv=3,method="decision_function")

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train, y_scores)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train, y_scores)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, home_credit_prepared, y_train, cv=3,method="predict_proba")

In [ ]:
forest_clf.fit(home_credit_prepared, y_train)

In [ ]:
y_scores_forest = y_probas_forest[:, 1] # score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train,y_scores_forest)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
roc_auc_score(y_train, y_scores_forest)

In [ ]:
y_scores_forest.shape

### Submissions

In [ ]:
submission_data = os.path.join('.','data','application_test.csv')
submission_df = pd.read_csv(submission_data)

submission_X = submission_df.drop('SK_ID_CURR', axis=1) # drop labels for training set
submission_ids = submission_df['SK_ID_CURR'].copy()

In [ ]:
import random

def set_unknown(colVal, setOfValues):
    if colVal in setOfValues:
        return colVal
    return random.choice(list(setOfValues))
    

In [ ]:
# here is how you would use the above function to fill values in the test set
# that are not see in the training set

practice = DataFrame({'col1':['cat','cat','dog','pig'], 'col2':['cat','cat','dog','pig']})
values = set(['cat','dog'])

list(np.vectorize(set_unknown)(practice['col1'], values))

In [ ]:
# apply the column cleaning logic to an entire dataframe

def clean_submission_categories(trainDF, submissionDF, categoricalColumns):
    X_submission_new = {}
    for name, values in submissionDF[categoricalColumns].iteritems():
        uniqueColumnValues = set(trainDF[name])
        currentValues = np.vectorize(set_unknown)(submissionDF[name], uniqueColumnValues)
        X_submission_new[name] = currentValues
    return X_submission_new

In [ ]:
submissionCategoricals = clean_submission_categories(X_train, submission_X, categoricalColumns)

submissionCategoricalsDF = pd.DataFrame.from_dict(submissionCategoricals)

In [ ]:
# check the training data has the same number of columns as the test data

print(f'Train numerical columns: {applicationDF[numericalColumns].shape[1]} columns')
print(f'Train categorical columns: {applicationDF[categoricalColumns].shape[1]} columns')
print(f'Test numerical columns: {submission_df[numericalColumns].shape[1]} columns')
print(f'Test categorical columns: {submissionCategoricalsDF.shape[1]} columns')

In [ ]:
finalSubmissionDF = pd.concat([submission_df[numericalColumns], submissionCategoricalsDF], axis=1)

In [ ]:
submission_prepared = full_pipeline.transform(finalSubmissionDF)

In [ ]:
y_probas_forest = forest_clf.predict_proba(submission_prepared)

In [ ]:
sub

In [ ]:
finalOutput = DataFrame(np.concatenate([submission_ids.values.reshape(-1,1), y_probas_forest[:,1:]], axis=1))

finalOutput.loc[:,0] = finalOutput.loc[:,0].astype(int)

finalOutput.columns=['SK_ID_CURR','TARGET']

In [ ]:
finalOutput.to_csv('submissions/submission.csv', index=False)

In [ ]:
# y_sgd_submission = sgd_clf.predict_proba(submission_prepared)

### Questions

1. How to give a neural network categorical data?

### Playground

In [ ]:
!ls -ltr data/

In [ ]:
bureauBalancePath = os.path.join('.','data', 'bureau_balance.csv')

df = pd.read_csv(bureauBalancePath)

df.head()

In [ ]:
df.STATUS.value_counts()

In [ ]:
bureauBalancePath = os.path.join('.','data', 'bureau.csv')

df = pd.read_csv(bureauBalancePath)

df.head()

### Ideas
1. 